In [20]:
import numdifftools as nd
import numpy as np
import sympy
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
def F(x):
    x_1 = x[0]
    x_2 = x[1]
    # return 10 * x_1 ** 2 + x_2 ** 2 # 0 (0, 0)
    # return 11 * x_1 ** 2 + 4 * x_2 ** 2 # 0 (0, 0)
    return 34 * x_1 ** 2 + x_2 ** 2 # 0 (0, 0)

In [3]:
def Fp(f, x):
    return f.subs('tk', x)


def paul(f, x_1 = 1, delta_x = 1, e_1 = 0.003, e_2 = 0.03):
    n_val = 0
    step = 2

    while (True):
        if step != 6:
            x_2 = x_1 + delta_x
            f_1, f_2 = Fp(f, x_1), Fp(f, x_2)
            if f_1 > f_2:
                x_3 = x_1 + 2 * delta_x
            else:
                x_3 = x_1 - delta_x
            f_3 = Fp(f, x_3)
            n_val += 3
        f_min = min([f_1, f_2, f_3])
        if f_min == f_1:
            x_min = x_1
        elif f_min == f_2:
            x_min = x_2
        elif f_min == f_3:
            x_min = x_3
        znam = ((x_2 - x_3) * f_1 + (x_3 - x_1) * f_2 + (x_1 - x_2) * f_3)
        if znam == 0:
            x_1 = x_min
            continue
        x_ = 1/2 * (
            ((x_2 ** 2 - x_3 ** 2) * f_1 + (x_3 ** 2 - x_1 ** 2) *
            f_2 + (x_1 ** 2 - x_2 ** 2) * f_3) /
            znam
        )
        f_ = Fp(f, x_)  
        n_val += 1
        first_module = abs((f_min - f_) / f_)
        second_module = abs((x_min - x_) / x_)
        if first_module < e_1 and second_module < e_2:
            x_star = x_
            step = 2
            break
        elif x_1 <= x_ <= x_3:

            step = 6

            if x_2 < x_:
                x_1 = x_2
            elif x_2 >= x_:
                x_3 = x_2
            x_2 = x_
            f_1, f_2, f_3 = Fp(f, x_1), Fp(f, x_2), Fp(f, x_3)
            n_val += 3
        else:
            x_1 = x_
            step = 2
    return x_star, n_val

In [4]:
def gradient_fall(x_0 = (0.5, 1), e = 0.1, e_1 = 0.1, e_2 = 0.15, M = 10):
    N = 0
    step = 0
    t = [0]
    while True:
        if step != 7:
            if step != 3:
                k = 0
            grad = nd.Gradient(F)(x_0[k])
            norm = np.linalg.norm(np.array(grad))
            if norm < e_1:
                x_res = x_0[k]
                break
            if k >= M:
                x_res = x_0[k]
                break
            try:
                if len(t) < 2:
                    t[k] = 0.5
                else:
                    t[k] = t[k-1]
            except:
                t.append(t[k-1])
        try:
            x_0[k + 1] = (np.array(x_0[k]) - t[k] * np.array(grad))
        except IndexError:
            x_0.append(np.array(x_0[k]) - t[k] * np.array(grad))
        f_k_1 = F(x_0[k+1])
        f_k = F(x_0[k])
        N += 2
        if f_k_1 - f_k < 0 or f_k_1 - f_k < -e * norm * norm: # or f_k_1 - f_k < -e * norm * norm
            step = 9
        else:
            t[k] = t[k]/2
            step = 7
            continue
        norm_x = np.linalg.norm(np.array(x_0[k+1]) - np.array(x_0[k]))
        abs_f_x = abs(F(x_0[k+1]) - F(x_0[k]))
        norm_x_1 = np.linalg.norm(np.array(x_0[k]) - np.array(x_0[k-1]))
        abs_f_x_1 = abs(F(x_0[k]) - F(x_0[k-1]))
        N += 4
        if norm_x < e_2 and abs_f_x < e_2 and abs_f_x_1 < e_2 and norm_x_1 < e_2:
            x_res = x_0[k+1]
            break
        else:
            k += 1
            step = 3
    return x_res, N

In [5]:
def quick_gradient_fall(x_0 = [[0.5, 1]], e_1 = 0.1, e_2 = 0.15, M = 10):
    N = 0
    step = 0
    t = [0]
    while True:

        if step != 3:
            k = 0
        grad = nd.Gradient(F)(np.array(x_0[k], dtype=np.float64))
        norm = np.linalg.norm(np.array(grad))
        if norm < e_1:
            x_res = x_0[k]
            break
        if k >= M:
            x_res = x_0[k]
            break
        tk = sympy.symbols('tk')
        try:
            x_0[k + 1] = (np.array(x_0[k]) - tk * np.array(grad))
        except IndexError:
            x_0.append(np.array(x_0[k]) - tk * np.array(grad))
        f = 2*x_0[k+1][0]**2 + x_0[k+1][0] * x_0[k+1][1] + 3 * x_0[k+1][1]**2
        try:
            t[k], _ = paul(f)
        except:
            a, _ = paul(f)
            t.append(a)
        f_k_1 = F(x_0[k+1])
        f_k = F(x_0[k])
        N += 2
        x_0[k + 1][0] = x_0[k + 1][0].subs('tk', t[k])
        x_0[k + 1][1] = x_0[k + 1][1].subs('tk', t[k])
        x_0[k + 1] = list(x_0[k + 1])
        norm_x = np.linalg.norm(np.array(x_0[k+1], dtype=np.float64) - np.array(x_0[k], dtype=np.float64))
        abs_f_x = abs(F(x_0[k+1]) - F(x_0[k]))
        norm_x_1 = np.linalg.norm(np.array(x_0[k], dtype=np.float64) - np.array(x_0[k-1], dtype=np.float64))
        abs_f_x_1 = abs(F(x_0[k]) - F(x_0[k-1]))
        N += 4
        if norm_x < e_2 and abs_f_x < e_2 and abs_f_x_1 < e_2 and norm_x_1 < e_2:
            x_res = x_0[k+1]
            break
        else:
            k += 1
            step = 3
    return x_res, N

In [46]:
speeds = []
errors = []

In [40]:
def main():
    true_x = (0, 0)
    true_f_x = 0

    x_0 = [[0.5, 1]]
    e = 0.1
    e_1 = 0.1
    e_2 = 0.15
    M = 10

    print(f"Истинный минимум в x = {true_x}, f(x) = {true_f_x}")
    print(f"При параметрах x_0 = {x_0}, e = {e}, e_1 = {e_1}, e_2 = {e_2}, M = {M}")
    start = time.time()
    x, N = gradient_fall(x_0 = x_0, e_1 = e_1, e_2 = e_2, M = M)
    end = time.time()
    f_x = F(x)
    print(f"Минимум алгоритма в x = {x}, f(x) = {f_x}\nКоличество операций: {N}\nОтклонение +-{abs(true_f_x - f_x)}")
    print(f"Время выполнения: {end - start}\n")
    speeds.append(end - start)
    errors.append(abs(true_f_x - f_x))

    print(f"Истинный минимум в x = {true_x}, f(x) = {true_f_x}")
    print(f"При параметрах x_0 = {x_0}, e = {e}, e_1 = {e_1}, e_2 = {e_2}, M = {M}")
    start = time.time()
    x, N = quick_gradient_fall(x_0 = x_0, e_1 = e_1, e_2 = e_2, M = M)
    end = time.time()
    f_x = F(x)
    print(f"Минимум алгоритма в x = {x}, f(x) = {f_x}\nКоличество операций: {N}\nОтклонение +-{abs(true_f_x - f_x)}")
    print(f"Время выполнения: {end - start}\n")
    speeds.append(end - start)
    errors.append(abs(true_f_x - f_x))

In [48]:
main()

Истинный минимум в x = (0, 0), f(x) = 0
При параметрах x_0 = [[0.5, 1]], e = 0.1, e_1 = 0.1, e_2 = 0.15, M = 10
Минимум алгоритма в x = [-1.22070313e-04  9.09149170e-01], f(x) = 0.8265527198091149
Количество операций: 28
Отклонение +-0.8265527198091149
Время выполнения: 0.010985612869262695

Истинный минимум в x = (0, 0), f(x) = 0
При параметрах x_0 = [[0.5, 1], array([-0.03125,  0.96875]), array([0.00195312, 0.93847656]), array([-1.22070313e-04,  9.09149170e-01])], e = 0.1, e_1 = 0.1, e_2 = 0.15, M = 10
Минимум алгоритма в x = [-0.000714537999864243, 0.0103545989023509], f(x) = 0.000124576913239066
Количество операций: 54
Отклонение +-0.000124576913239066
Время выполнения: 0.4120142459869385

